In [1]:
import pickle
import pandas as pd
import numpy as np
from load_data import *
from beam_search_module import *

In [2]:

stock_df = make_growth_target_df('datasets/stock_data_for_emm.pkl')
stock_df_index = stock_df.copy()
stock_df.drop(['index'], inplace=True, axis=1)


In [3]:
stock_df_index

,index,country,industry,currency,exchangeTimezoneName,exchange,sector,averageVolume10days,enterpriseToEbitda,marketCap,debtToEquity,fullTimeEmployees,growth_target
0,SOG.PA,France,Aerospace & Defense,EUR,Europe/Paris,PAR,Industrials,382.0,11.822,5.750674e+07,50.783,1102.0,"[0.0, 1.17, 2.41, 0.41, 2.65, -1.39, -7.24, -4..."
1,AOF.DE,Germany,Software—Application,EUR,Europe/Berlin,GER,Technology,8329.0,33.294,1.241838e+09,32.492,650.0,"[0.0, -1.84, 17.01, 3.7, -8.24, 8.01, -3.6, 4...."
2,LFG.MI,Italy,Entertainment,EUR,Europe/Rome,MIL,Communication Services,330.0,-21.050,2.849466e+07,181.181,34.0,"[0.0, -7.96, -2.61, 0.89, 13.94, -4.85, -6.12,..."
3,ENV.MI,Italy,Packaged Foods,EUR,Europe/Rome,MIL,Consumer Defensive,3831.0,8.794,5.971590e+07,68.513,232.0,"[0.0, -3.39, 0.58, -10.17, 11.33, -2.91, 2.1, ..."
4,FPO.F,United Kingdom,Other Precious Metals & Mining,EUR,Europe/Berlin,FRA,Basic Materials,108.0,1.402,2.123667e+08,81.212,3474.0,"[0.0, 8.5, -15.79, 5.28, 2.24, -1.94, 7.5, -1...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9774,MEL.L,Japan,Electrical Equipment & Parts,JPY,Europe/London,LSE,Industrials,4893.0,6.287,2.079737e+10,10.654,145696.0,"[0.0, 1.08, 10.44, -11.75, -11.15, 7.96, -9.17..."
9775,RICO.L,Japan,Business Equipment & Supplies,JPY,Europe/London,LSE,Industrials,1940.0,5.531,5.086319e+09,35.268,78360.0,"[0.0, 5.82, 1.71, 7.84, -10.55, 3.58, -7.94, 2..."
9776,KNM.L,Japan,Electronic Gaming & Multimedia,JPY,Europe/London,LSE,Communication Services,3530.0,7.976,6.297095e+09,18.311,4894.0,"[0.0, 6.5, 0.0, -10.85, -1.95, -3.62, -3.95, 9..."
9777,MAW.L,Japan,Electronic Components,JPY,Europe/London,LSE,Technology,350.0,7.200,1.774490e+09,1.943,1297.0,"[0.0, 0.0, 0.0, 0.0, 18.86, 0.0, 0.0, 0.0, 10...."


In [4]:
data = stock_df.values.tolist()
column_names = list(stock_df.columns)
beam_width = 10
beam_depth = 3
nr_bins = 8
nr_saved = 10
subgroup_size = len(data) * 0.05
target = 'growth_target'
window_size = 5
target_ind = column_names.index(target)
all_time_series = [i[target_ind] for i in data]
all_time_series = np.array(all_time_series)
targets_baseline = np.mean(all_time_series, axis=0)
att_indices = list(range(0, len(column_names)))
att_indices.remove(target_ind)
att_columns = [column_names[i] for i in att_indices]
types = categorize_columns_in_order(stock_df, att_columns)

In [5]:
results = beam_search(data, targets_baseline, column_names, beam_width, beam_depth, nr_bins, nr_saved,
                      subgroup_size, target, types, window_size)

print(results)

[(313.6180541798598, (('marketCap', 730348608.0, <function gt at 0x000001E4C4F59FC0>), ('fullTimeEmployees', 1100.0, <function gt at 0x000001E4C4F59FC0>), ('sector', 'Energy', <function neq at 0x000001E4C4F5A170>)), 5122), (312.0764297243845, (('marketCap', 730348608.0, <function gt at 0x000001E4C4F59FC0>), ('sector', 'Energy', <function neq at 0x000001E4C4F5A170>), ('fullTimeEmployees', 1282.0, <function gt at 0x000001E4C4F59FC0>)), 5052), (301.477281621563, (('marketCap', 730348608.0, <function gt at 0x000001E4C4F59FC0>), ('industry', 'Oil & Gas Integrated', <function neq at 0x000001E4C4F5A170>), ('fullTimeEmployees', 1093.0, <function gt at 0x000001E4C4F59FC0>)), 5294), (301.3663285480391, (('marketCap', 730348608.0, <function gt at 0x000001E4C4F59FC0>), ('fullTimeEmployees', 1100.0, <function gt at 0x000001E4C4F59FC0>), ('industry', 'Oil & Gas Integrated', <function neq at 0x000001E4C4F5A170>)), 5284), (300.147666821288, (('marketCap', 730348608.0, <function gt at 0x000001E4C4F59FC

In [6]:
descriptors = results[0][1]
df = filter_df_on_descriptors(stock_df_index, descriptors)
df

,index,country,industry,currency,exchangeTimezoneName,exchange,sector,averageVolume10days,enterpriseToEbitda,marketCap,debtToEquity,fullTimeEmployees,growth_target
5,M07.F,United States,Security & Protection Services,EUR,Europe/Berlin,FRA,Industrials,0.0,14.054,5.451780e+09,71.777,4800.0,"[0.0, -8.54, -4.13, 6.08, -0.78, 7.8, 11.86, 2..."
8,I8MA.F,Australia,Steel,EUR,Europe/Berlin,FRA,Basic Materials,0.0,3.925,1.864038e+09,27.653,4071.0,"[0.0, 14.86, 4.23, 3.94, -19.32, 12.26, 2.55, ..."
9,AUL.F,Italy,Restaurants,EUR,Europe/Berlin,FRA,Consumer Cyclical,0.0,14.216,1.727039e+09,235.090,34000.0,"[0.0, 6.94, -2.49, -6.89, -1.15, 2.04, -0.57, ..."
10,CE9.F,Canada,Aerospace & Defense,EUR,Europe/Berlin,FRA,Industrials,0.0,17.224,6.691759e+09,77.800,13000.0,"[0.0, 5.04, -4.55, 2.51, -2.71, 5.97, 16.59, -..."
11,PUP.F,United States,REIT—Industrial,EUR,Europe/Berlin,FRA,Real Estate,14.0,22.204,5.064935e+10,73.056,5800.0,"[0.0, -1.22, -11.15, 1.41, 3.92, 1.79, 8.91, 8..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9774,MEL.L,Japan,Electrical Equipment & Parts,JPY,Europe/London,LSE,Industrials,4893.0,6.287,2.079737e+10,10.654,145696.0,"[0.0, 1.08, 10.44, -11.75, -11.15, 7.96, -9.17..."
9775,RICO.L,Japan,Business Equipment & Supplies,JPY,Europe/London,LSE,Industrials,1940.0,5.531,5.086319e+09,35.268,78360.0,"[0.0, 5.82, 1.71, 7.84, -10.55, 3.58, -7.94, 2..."
9776,KNM.L,Japan,Electronic Gaming & Multimedia,JPY,Europe/London,LSE,Communication Services,3530.0,7.976,6.297095e+09,18.311,4894.0,"[0.0, 6.5, 0.0, -10.85, -1.95, -3.62, -3.95, 9..."
9777,MAW.L,Japan,Electronic Components,JPY,Europe/London,LSE,Technology,350.0,7.200,1.774490e+09,1.943,1297.0,"[0.0, 0.0, 0.0, 0.0, 18.86, 0.0, 0.0, 0.0, 10...."
